In [45]:
from scipy.interpolate import interp1d
import numpy as np

import shlex
import subprocess as sp

import prettyplotlib as ppl
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import io
from pandas import read_csv
import os

%matplotlib inline


os.chdir("/home/matheus/Documentos/Mestrado/teste/poc-test-user-github/")

cmd = 'git log --no-merges --no-renames --numstat --pretty=format:"%x09%x09%x09%aN"'
p = sp.Popen(shlex.split(cmd), stdout=sp.PIPE)

stdout, _ = p.communicate()

table = read_csv(io.StringIO(stdout.decode('utf-8')), sep="\x09", header=None,names=['additions', 'deletions', 'path','author', 'test'])

# convert to DataFrame
commit_data = table[['additions', 'deletions', 'path']].join(table[['author']].fillna(method='ffill')).dropna()
commit_data.head()

commit_data.to_csv("testeP.csv")


## Identifica quais são os arquivos de Test

In [46]:
commit_data.filter(items=['path', 'author'])
#marca somente os paths que estão dentro da pasta /test/

commit_data['origin'] = commit_data['path']
commit_data['test'] = commit_data.origin.apply(lambda x: 1 if '/test/' in x else 0)
commit_data['path'] = commit_data.path.apply(lambda x: x.split('.')[-1])
commit_data

,additions,deletions,path,author,origin,test
1,14.0,0.0,html,Matheus Flauzino,frontend/index.html,0
2,0.0,0.0,css,Matheus Flauzino,frontend/style.css,0
4,2.0,0.0,js,Leticia Amaral,backend/services.js,0
5,1.0,0.0,js,Leticia Amaral,backend/test/http.test.js,1
7,4789.0,180.0,json,Matheus Flauzino,backend/package-lock.json,0
8,3.0,0.0,json,Matheus Flauzino,backend/package.json,0
9,1.0,0.0,js,Matheus Flauzino,backend/test/userTest.js,1
10,1.0,0.0,js,Matheus Flauzino,backend/tests/user.test.js,0
12,26.0,0.0,json,Leticia Amaral,backend/package-lock.json,0
13,1.0,0.0,json,Leticia Amaral,backend/package.json,0


In [47]:
#amount  files java in total 
total=commit_data.path.str.contains(r'js$',regex=True).sum()
total

7

In [24]:
amount_test = commit_data.test.sum()
amount_test

0

## Separa teste dos demais arquivos (quantitativo)

In [25]:
commit_data['test'].value_counts()

0    6
Name: test, dtype: int64

## Distribuição por arquivos

In [26]:
commit_data['path'].value_counts()

gitignore    2
json         2
js           2
Name: path, dtype: int64

## Somente linguagem Java

In [102]:
java_all = commit_data[commit_data['path'] == 'java']
java_all

,additions,deletions,path,author,origin,test
5,23,11,java,Jiacheng Liu,integration/tools/validation/src/main/java/all...,0
6,15,0,java,Jiacheng Liu,integration/tools/validation/src/test/java/all...,1
8,15,8,java,Göktürk Gezer,core/server/common/src/test/java/alluxio/maste...,1
10,2,2,java,Bradley Yoo,core/server/master/src/main/java/alluxio/web/M...,0
11,2,2,java,Bradley Yoo,core/server/worker/src/main/java/alluxio/web/W...,0
...,...,...,...,...,...,...
130822,646,0,java,haoyuan,src/main/java/tachyon/thrift/PartitionInfo.java,0
130823,382,0,java,haoyuan,src/main/java/tachyon/thrift/SuspectedPartitio...,0
130824,5824,0,java,haoyuan,src/main/java/tachyon/thrift/WorkerService.java,0
130825,24,0,java,haoyuan,src/test/java/tachyon/TestUserInfo.java,1


## Quantidade de contribuição Java por author

In [103]:
java_all['author'].value_counts()

Jiri Simsa         12073
Bin Fan             9734
Calvin Jia          7696
Andrew Audibert     7276
Gene Pang           3898
                   ...  
YinQiaoyi              1
suinking               1
nimrodzhang            1
xuliang                1
xiaoshankeji           1
Name: author, Length: 1005, dtype: int64

In [95]:
count_java = java_all['path'].count()
count_java

83504

In [96]:
java_test_all = java_all[java_all['test'] == 1]
java_test_all

,additions,deletions,path,author,origin,test
6,15,0,java,Jiacheng Liu,integration/tools/validation/src/test/java/all...,1
8,15,8,java,Göktürk Gezer,core/server/common/src/test/java/alluxio/maste...,1
96,7,8,java,Zac Blanco,core/server/master/src/test/java/alluxio/maste...,1
97,1,1,java,Zac Blanco,core/server/master/src/test/java/alluxio/maste...,1
98,2,2,java,Zac Blanco,tests/src/test/java/alluxio/client/fs/MultiUfs...,1
...,...,...,...,...,...,...
130116,0,2,java,haoyuan,src/main/java/tachyon/test/TestReadBig.java,1
130337,2,0,java,haoyuan,src/main/java/tachyon/test/TestReadBig.java,1
130426,34,0,java,haoyuan,src/main/java/tachyon/test/TestReadBig.java,1
130825,24,0,java,haoyuan,src/test/java/tachyon/TestUserInfo.java,1


In [101]:
count_java_test = java_test_all['path'].count()
count_java_test

21918

## Quantidade de teste que cada Author fez

In [100]:
java_test_all['author'].value_counts()

Jiri Simsa         3438
Bin Fan            2820
Calvin Jia         2348
Andrew Audibert    2318
Haoyuan Li          865
                   ... 
courseralxy           1
qgxiaozhan            1
tangjijun             1
Joney-Yf              1
Kevin Chen            1
Name: author, Length: 457, dtype: int64

In [12]:
#BY author  by Author  Java

author = commit_data[commit_data['path'].str.contains('java') & commit_data['author'].str.contains('Andrew Audibert')]
#author=commit_data.author.str.contains(r'Andrew Audibert').sum()
author=author.author.str.contains('Andrew Audibert').sum()
author

7276

In [13]:
Java=round((author/ total)*100, 2)
Java

8.71

In [14]:
#amount  files GO in total 
total=commit_data.path.str.contains(r'go').sum()
total

244

In [15]:
#amount  files GO by Author 
author = commit_data[commit_data['path'].str.contains('go') & commit_data['author'].str.contains('Andrew Audibert')]
#author=commit_data.author.str.contains(r'Andrew Audibert').sum()
author=author.author.str.contains('Andrew Audibert').sum()
author

103

In [17]:
#Export results 
commit_data.to_csv("resultt.csv",index=False)

#amount  files CSS in total 
total=commit_data.path.str.contains(r'css').sum()
total


186

In [21]:
#amount  files CSS by Author 
author = commit_data[commit_data['path'].str.contains('css') & commit_data['author'].str.contains('Andrew Audibert')]
#author=commit_data.author.str.contains(r'Andrew Audibert').sum()
author=author.author.str.contains('Andrew Audibert').sum()
author

1